In [1]:
import requests
import json

In [13]:
first = requests.get('http://localhost:8000/movie_detail/4/')

In [16]:
second = requests.get('http://localhost:8000/movie_list/')

In [50]:
print(second.content)
# dir(second)


b'<!DOCTYPE html>\n<html>\n<head lang="en">\n    <meta charset="UTF-8">\n    <title></title>\n</head>\n\n<body>\n<div style="text-align: center">\n    <a href="/">Home</a>\n    <a href="/movie_list/">List</a>\n    <a href="/create_movie/">Create</a>\n</div>\n\n\n<div style="padding-top: 30px; text-align: center;">\n    \n        <div><a href="/movie_detail/4/">King Kong</a>\n            <form method="POST" action="/delete_movie/4/">\n                <input type=\'hidden\' name=\'csrfmiddlewaretoken\' value=\'E3ZslLbV3ENaIZCFZEb6utidaMNCGeN5\' />\n                <button type="submit">Delete</button>\n            </form>\n            <form method="GET" action="/update_movie/4/">\n                <input type=\'hidden\' name=\'csrfmiddlewaretoken\' value=\'E3ZslLbV3ENaIZCFZEb6utidaMNCGeN5\' />\n                <button type="submit">Update</button>\n            </form>\n        </div>\n    \n        <div><a href="/movie_detail/5/">Somebody to Love</a>\n            <form method="POST" actio

In [37]:
help(first.content)

Help on bytes object:

class bytes(object)
 |  bytes(iterable_of_ints) -> bytes
 |  bytes(string, encoding[, errors]) -> bytes
 |  bytes(bytes_or_buffer) -> immutable copy of bytes_or_buffer
 |  bytes(int) -> bytes object of size given by the parameter initialized with null bytes
 |  bytes() -> empty bytes object
 |  
 |  Construct an immutable array of bytes from:
 |    - an iterable yielding integers in range(256)
 |    - a text string encoded using the specified encoding
 |    - any object implementing the buffer API.
 |    - an integer
 |  
 |  Methods defined here:
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __contains__(self, key, /)
 |      Return key in self.
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __getitem__(self, key, /)
 |      Return self[key].
 |  
 |  __getnewargs__(...)
 |  
 |  __g

In [48]:
serialized_info = json.loads(str(first.content))

ValueError: Expecting value: line 1 column 1 (char 0)

In [42]:
from html import parser

In [43]:

class HTMLtoJSONParser(parser.HTMLParser):
    def __init__(self, raise_exception = True) :
        html.parser.HTMLParser.__init__(self)
        self.doc  = { }
        self.path = []
        self.cur  = self.doc
        self.line = 0
        self.raise_exception = raise_exception
         
    @property
    def json(self):
        return self.doc
         
    @staticmethod
    def to_json(content, raise_exception = True):
        parser = HTMLtoJSONParser(raise_exception = raise_exception)
        parser.feed(content)
        return parser.json
         
    def handle_starttag(self, tag, attrs):
        self.path.append(tag)
        attrs = { k:v for k,v in attrs }
        if tag in self.cur :
            if isinstance(self.cur[tag],list) :
                self.cur[tag].append(  { "__parent__": self.cur } )
                self.cur = self.cur[tag][-1]
            else :
                self.cur[tag] = [ self.cur[tag] ]
                self.cur[tag].append(  { "__parent__": self.cur } )
                self.cur = self.cur[tag][-1]
        else :
            self.cur[tag] = { "__parent__": self.cur }
            self.cur = self.cur[tag]
             
        for a,v in attrs.items():
            self.cur["#" + a] = v
        self.cur[""] = ""
                 
    def handle_endtag(self, tag):
        if tag != self.path[-1] and self.raise_exception :
            raise Exception("html is malformed around line: {0} (it might be because of a tag <br>, <hr>, <img .. > not closed)".format(self.line))
        del self.path[-1]
        memo = self.cur
        self.cur = self.cur["__parent__"]
        self.clean(memo)
                 
    def handle_data(self, data):
        self.line += data.count("\n")
        if "" in self.cur :
            self.cur[""] += data
             
    def clean(self, values):
        keys = list(values.keys())
        for k in keys:
            v = values[k]
            if isinstance(v, str) :
                #print ("clean", k,[v])
                c = v.strip(" \n\r\t")
                if c != v : 
                    if len(c) > 0 : 
                        values[k] = c
                    else : 
                        del values[k]
        del values["__parent__"]

In [60]:
content = '<html><body><div class="an_example"><p>one paragraph</p></div></body></html>'
js = HTMLtoJSONParser.to_json(ap)
print (js)

Exception: html is malformed around line: 5 (it might be because of a tag <br>, <hr>, <img .. > not closed)

In [56]:
print(first.text)

<!DOCTYPE html>
<html>
<head lang="en">
    <meta charset="UTF-8">
    <title></title>
</head>

<body>
<div style="text-align: center">
    <a href="/">Home</a>
    <a href="/movie_list/">List</a>
    <a href="/create_movie/">Create</a>
</div>


<a href="/movie_list/">Home</a>
<h1> King Kong </h1>




</body>
</html>


In [58]:
ap = '''<!DOCTYPE html>
<html>
<head lang="en">
    <meta charset="UTF-8">
    <title></title>
</head>

<body>
<div style="text-align: center">
    <a href="/">Home</a>
    <a href="/movie_list/">List</a>
    <a href="/create_movie/">Create</a>
</div>


<a href="/movie_list/">Home</a>
<h1> King Kong </h1>




</body>
</html>'''